# COMP3411/9418 21T0 Assignment 1

- Lecturer: Anna Trofimova
- School of Computer Science and Engineering, UNSW Sydney
- Last Update 13th January at 18:00pm, 2021
$$
% macros
\newcommand{\indep}{\perp \!\!\!\perp}
$$

## Student:
Hoya Lee, z5226463

### Submission
This interactive notebook contains the instructions to complete assignment 1; You should submit this notebook with the code and answers in one single file in .ipybn format with the name assignment1.ipybn. **Write your name and zID in the cell above** (to edit the markdown text double-click the cell).

There is a maximum file size cap of 5MB, so make sure your submission does not exceed this size. The submitted notebook should contain all your source code and answers. You can add new cells and use markdown text to organise and explain your implementation/answer.

Submit your files using give. On a CSE Linux machine, type the following on the command-line:

The submission deadline is **22nd January at 11.59pm, 2021.**

### Late Submission Policy 
The penalty is set at 20% per late day. This is ceiling penalty, so if your submission is marked 60/100 and it was submitted two days late, you still get 60/100. If you submit 5 days later, then the penalty is 100% and your mark will be 0.

### Plagiarism
This is an individual assignment. Remember that **all** work submitted for this assignment must be your own **individual** work and no code
sharing or copying is allowed. You may use code from the Internet only with suitable attribution
of the source in your program. **Do not use public code repositories as your code might be copied.** Keep in mind that sharing parts of assignment solutions is a form of plagiarism. All submitted assignments will be run through plagiarism detection software to detect similarities to other submissions. You should carefully read the UNSW policy on academic integrity and plagiarism. 

### **Note: if your life is falling apart and you are tempted to plagiarize as to not fail the assignment, please send me an email instead. I would rather grant you an extention than report you to the Student Conduct and Integrity Unit**.

### Description

This assignment consists of two problems presented in a puzzle game context. For the first problem, you will be implementing a path search problem along with common search algorithms and evaluating their properties. For the second problem, you will be exploring how to define and solve a constraint satisfaction problem. To implement the solutions, you have to use AIPython code that provides you with many implemented classes and examples of their use.  

When working with a Jupyter notebook, you can edit the \*.py files either in the Jupyter interface (in your browser) or with your favorite editor (e.g., PyCharm). Whenever you save a \*.py file, the notebook will reload their content directly.

The libraries that you can use (and need) in this assignment are  the following (run the cell below to import the libraries):

In [2]:
import time
import numpy as np
import sys
sys.path.insert(0, "aipython")
from aipython.searchGeneric import *
from aipython.searchProblem import *
from aipython.cspProblem import *
from aipython.cspSearch import *

*Tip: If you placed the notebook correctly it should not give you any errors.*

*If you think that there are other libraries you might need then send an email to the lecturer-in-charge to confirm.*

### Part 1 - Heuristics [5 marks]

This part of the assignment is based on a popular puzzle, Game of Fifteen, also known as 15-puzzle. The game consists of 15 numbered tiles on a four by four grid where one tile is missing. To solve the puzzle, the tiles must be moved so that they are ordered from 1 to 15. The goal state of the puzzle is defined as a multidimensional array of digits#, where 0 represents the missing tile, as follow:

In [3]:
goal = [[1, 2, 3, 4], 
        [5, 6, 7, 8], 
        [9, 10, 11, 12], 
        [13, 14, 15, 0]]

#### Task 1.1
Edit the code below to implement a search problem class representing the Game of Fifteen that uses heuristic based on Manhattan Distance (h1). The class must be implemented by extending the class Search\_problem. 

Manhattan Distance between two points $p1$ at $(x1, y1)$ and $p2$ at $(x2, y2)$ is $d_{M}(p1, p2) = |x1 - x2| + |y1 - y2|$

*Tip: If you have problems understanding how to overwrite methods take a look at the implementation of Search_problem_from_explicit_graph class in searchProblem.py*


In [4]:
class GameFifteenProblem(Search_problem):
    
    def __init__(self, start, goal):
        self.start = start
        self.goal = goal
        return
    
    def start_node(self):
        """Returns the start node"""
        return self.start
    
    def is_goal(self, node):
        """Returns True if the node is the goal, otherwise False"""
        for i in range(len(node)):
            for j in range(len(node[i])):
                if(node[i][j] != self.goal[i][j]):
                    return False
        return True
    
    def neighbors(self, node):
        """Returns a list of the arcs for the neighbors of node, for example: 
        return [Arc(node, to_neighbor_node1, cost1), Arc(node, to_neighbor_node2, cost2)]"""
        arcs = []
        n = np.copy(node)
        
        i = np.argwhere(n == 0)[0][0]
        j = np.argwhere(n == 0)[0][1]
        if j > 0:
            copy = np.copy(node)
            copy[i][j] = copy[i][j - 1]
            copy[i][j - 1] = 0       
            arcs.append(Arc(node, copy, 1))   
        if j < 3:
            copy = np.copy(node)
            copy[i][j] = copy[i][j + 1]
            copy[i][j + 1] = 0 
            arcs.append(Arc(node, copy, 1)) 
        if i > 0:
            copy = np.copy(node)
            copy[i][j] = copy[i - 1][j]
            copy[i - 1][j] = 0
            arcs.append(Arc(node, copy, 1)) 
            
        if i < 3:
            copy = np.copy(node)
            copy[i][j] = copy[i + 1][j]
            copy[i + 1][j] = 0       
            arcs.append(Arc(node, copy, 1)) 

       
        return arcs
    
    def heuristic(self, node):
        """Returns the heuristic value of the node 
        based on the Manhattan distance"""
        goal = np.copy(self.goal)
        n = np.copy(node)
        s = 0
        for idx in range(1,16):
            goalIdx = (np.argwhere(goal == idx))
            i = goalIdx[0][0]
            j = goalIdx[0][1]
            nodeIdx = np.argwhere(n == idx)
            x = nodeIdx[0][0]
            y = nodeIdx[0][1]
            s += abs(i - x) + abs(j - y)
        return s

To validate the correctness of the problem class use the A* searcher algorithm (from searchGeneric.py) to find a solution. 

*Tip: The cost of the solution should be 9.*

In [5]:
start = [[1, 2, 3, 4], 
         [9, 5, 6, 7], 
         [10, 11, 8, 0], 
         [13, 14, 15, 12]]
puzzle = GameFifteenProblem(start, goal)
searcher = AStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)

10 paths have been expanded and 22 paths remain in the frontier
Cost:  9


#### Task 1.2 
Implement search problem classes representing the Game of Fifteen that use heuristics based on Euclidean Distance (h2) and the number of the inversions of the permutation (h3). The classes must be implemented by extending the class GameFifteenProblem.

Euclidean distance between two points $p1$ at $(x1, y1)$ and $p2$ at $(x2, y2)$ is $d_{E}(p1, p2) =  \sqrt{(x1-x2)^{2}+ (y1-y2)^{2}}$

An inversion of a permutation $(t_{1},t_{2},...,t_{n})$ of the elements in a finite n-element set of positive integers $A=\{1,2,...,n\}$ is the pair $(t_{j},t_{k})$, where $j<k$ and $t_{j}>t_{k}$.

$\sum_{j=1}^{N} \sum_{k=j}^{N}    \{t_{j} > t_{k}: 1, otherwise: 0\}$, where $N$ is the total number of elements and $t_{i}$ is the value of i-th element.

In [6]:
class GameFifteenProblemEuclidean(GameFifteenProblem):
    def __init__(self, start, goal):
        (super().__init__(start, goal))

    def heuristic(self, node):
        """Returns the heuristic value of the node 
        based on the Euclidean distance"""
        goal = np.copy(self.goal)
        n = np.copy(node)
        s = 0
        for idx in range(1,16):
            goalIdx = (np.argwhere(goal == idx))
            i = goalIdx[0][0]
            j = goalIdx[0][1]
            nodeIdx = np.argwhere(n == idx)
            x = nodeIdx[0][0]
            y = nodeIdx[0][1]
            s += np.sqrt( abs(i - x) ** 2 + abs(j - y) ** 2)
        return s

In [15]:
class GameFifteenProblemInversions(GameFifteenProblem):
    def __init__(self, start, goal):
        (super().__init__(start, goal))

    def heuristic(self, node):
        """Returns the heuristic value of the node 
        based on the sum of the inversion number of a permutation"""
        goal = np.copy(self.goal)
        n = np.copy(node)
        flat = n.flatten()
        s = 0
        for j in range(1,16):
            k = j
            while k < 16:
                if flat[j] > flat[k]:
                    s += 1
                k += 1
            
        
        return s

#### Task 1.3
Run A* Search algorithm with every heuristic for the following three start states:

In [ ]:
# optimal path cost: 14
start14 = [[1, 2, 8, 3],
           [5, 6, 7, 4],
           [9, 15, 14, 11],
           [13, 10, 12, 0]]
puzzle = GameFifteenProblem(start14, goal)
searcher = AStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)

puzzle = GameFifteenProblemEuclidean(start14, goal)
searcher = AStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)

puzzle = GameFifteenProblemInversions(start14, goal)
searcher = AStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)

# optimal path cost: 17
start17 = [[1, 3, 6, 4],
           [5, 2, 8, 14],
           [9, 15, 7, 0],
           [13, 10, 12, 11]]
puzzle = GameFifteenProblem(start17, goal)
searcher = AStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)

puzzle = GameFifteenProblemEuclidean(start17, goal)
searcher = AStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)

puzzle = GameFifteenProblemInversions(start17, goal)
searcher = AStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)

# optimal path cost: 23
start23 = [[1, 3, 6, 4],
           [5, 8, 15, 14],
           [9, 2, 7, 0],
           [13, 10, 12, 11]]
puzzle = GameFifteenProblem(start23, goal)
searcher = AStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)

puzzle = GameFifteenProblemEuclidean(start23, goal)
searcher = AStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)

puzzle = GameFifteenProblemInversions(start23, goal)
searcher = AStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)


49 paths have been expanded and 94 paths remain in the frontier
Cost:  14
196 paths have been expanded and 390 paths remain in the frontier
Cost:  14
210332 paths have been expanded and 438160 paths remain in the frontier
Cost:  14
20 paths have been expanded and 45 paths remain in the frontier
Cost:  17
309 paths have been expanded and 704 paths remain in the frontier
Cost:  17


In each case record in the tables below the heuristic values for the start states, the number of the expanded nodes, and the costs of the solutions.

| Heuristic | h-value: start14 | h-value: start17 | h-value: start23 |
|-----------|------------------|------------------|------------------|
|Manhattan  | #                | #                | #                | 
|Euclidean  | #                | #                | #                | 
|Inversions | #                | #                | #                | 

| Heuristic | expanded: start14 | expanded: start17 | expanded: start23 |
|-----------|-------------------|-------------------|-------------------|
|Manhattan  | #                 | #                 | #                 | 
|Euclidean  | #                 | #                 | #                 |  
|Inversions | #                 | #                 | #                 |  

| Heuristic | path cost: start14 | path cost: start17 | path cost: start23 |
|-----------|--------------------|--------------------|--------------------|
|Manhattan  | #                  | #                  | #                  | 
|Euclidean  | #                  | #                  | #                  |
|Inversions | #                  | #                  | #                  |

Comment on the performance of the A* search algorithm with each heuristic based on the results in the tables. Explain which heuristic is better/worse and why.

### Part 2:  Search algorithms [7 marks]

*Tip: If you have problems understanding how to overwrite methods take a look at the implementation of AStarSearcher and  Searcher classes in searchGeneric.py*

*Tip: To initialize the frontier think of the type of structure used by the algorithm to store generated nodes. If it uses a priority queue use FrontierPQ from serachGeneric.py*

#### Task 2.1
Implement a class that performs Breadth First Search by extending class Searcher.

In [9]:
class BreadthFirstSearcher(Searcher):
    
    def __init__(self,  problem):
        super().__init__(problem)

    """ Initializes the forontier """
    def initialize_frontier(self):
        self.frontier = []
        return

    """ Returns True if there are no more nodes to expand """
    def empty_frontier(self):
        return len(self.frontier) == 0

    """ Adds the path to the forontier """
    def add_to_frontier(self, path):
        self.frontier.append(path)
        return
    
    """returns (next) path from the problem's start node
        to a goal node. """
    def search(self):
        visited = []
        while not self.empty_frontier():
            currPath = self.frontier.pop(0)
            if (currPath in visited):
                continue
            visited.append(currPath)
            self.display(2, "Expanding:",path,"(cost:",path.cost,")")
            if self.problem.is_goal(path.end()):    # solution found
                self.display(1, self.num_expanded, "paths have been expanded and",
                            len(self.frontier), "paths remain in the frontier")
                self.solution = path   # store the solution found
                return path
            else:
                neighs = self.problem.neighbors(path.end())
                self.display(3,"Neighbors are", neighs)
                for arc in reversed(list(neighs)):
                    self.add_to_frontier(Path(path,arc))
                self.display(3,"Frontier:",self.frontier)
        self.display(1,"No (more) solutions. Total of",
                     self.num_expanded,"paths expanded.")

    

#### Task 2.2

Implement a class that performs Iterative Deepening Search by extending class Searcher.

In [ ]:
class IterativeDeepeningSearcher(Searcher):
    def __init__(self, problem):
        super().__init__(problem)

    """ Initializes the forontier """
    def initialize_frontier(self):
        self.frontier = []
        return

    """ Returns True if there are no more nodes to expand """
    def empty_frontier(self):
        return len(self.frontier) == 0

    """ Adds the path to the forontier """
    def add_to_frontier(self, path):
        self.frontier.append(path)
        return
    
    def search(self):
        depth = 1
        while count > 1:
            
        while not self.empty_frontier():
            
        return

#### Task 2.3
Implement a class that performs Iterative Deepening A* Search by extending class Searcher.

In [ ]:
class IterativeDeepeningAStarSearcher(Searcher):
    """ Initializes the forontier """
    def initialize_frontier(self):
        self.frontier = []
        return

    """ Returns True if there are no more nodes to expand """
    def empty_frontier(self):
        return len(self.frontier) == 0

    """ Adds the path to the forontier """
    def add_to_frontier(self, path):
        self.frontier.append(path)
        return
    
    def search(self):
        maxDepth = 99999
        def depthLimitedSearch(src, maxDepth)
        for i in range(maxDepth):
            if (depthLimitedSearch)
        return

#### Task 2.4
Implement a class that performs Uniform Cost Search by extending class Searcher.

In [ ]:
class UniformCostSearcher(Searcher):
    """ Initializes the forontier """
    def initialize_frontier(self):
        this.frontier = FrontierPQ()
        return

    """ Returns True if there are no more nodes to expand """
    def empty_frontier(self):
        return this.frontier.empty()

    """ Adds the path to the forontier """
    def add_to_frontier(self, path):
        return this.frontier.add(path, path.cost)

#### Task 2.5

Run Breadth First Search (BFS), Iterative Deepenining Search (IDS), Iterative Deepening A* Search (IDA\*S), and Uniform Cost Search (UCS) algorithms on each of the 5 start states:

In [25]:
# optimal path cost: 10
start10 = [[2, 3, 7, 4],
           [1, 6, 11, 8],
           [5, 10, 0, 12],
           [9, 13, 14, 15]]

# optimal path cost: 24
start24 = [[2, 7, 11, 4],
           [6, 3, 12, 0],
           [1, 5, 15, 8],
           [9, 10, 13, 14]]

# optimal path cost: 30
start30 = [[2, 7, 11, 4],
           [6, 3, 12, 0],
           [1, 5, 15, 14],
           [9, 10, 8, 13]]

# optimal path cost: 36
start36 = [[7, 11, 12, 4],
           [2, 3, 14, 1],
           [6, 5, 13, 8],
           [9, 10, 15, 0]]

# optimal path cost: 41
start41 = [[7, 11, 12, 4],
           [2, 3, 8, 14],
           [10, 0, 5, 1],
           [6, 9, 13, 15]]

# your code

In each case, record in the table below the number of nodes generated during the search. If the algorithm runs out of memory or the number of nodes in the frontier exceeds 1 million items, just write “Mem” in your table. If the code runs for five minutes without producing output, terminate the process and write “Time” in your table.

*Tip: To edit the table double click the cell below.*

| Algorithm | 10      | 24      | 30      | 36      | 41      |
|-----------|---------|---------|---------|---------|---------|
|BFS        | #       | #       | #       | #       | #       |
|A*         | #       | #       | #       | #       | #       |
|IDS        | #       | #       | #       | #       | #       |
|IDA\*S     | #       | #       | #       | #       | #       |
|UCS        | #       | #       | #       | #       | #       |

Discuss the time and space efficiency of these four algorithms, comment on the results in the table.

### Part 3: Deceptive Starting States [3 marks]

#### Task 3.1 

Run IDA* on the starting states below and report the number of the expanded nodes.

In [26]:
start37_1 = [[7, 11, 12, 4],
             [2, 3, 14, 1],
             [6, 5, 13, 0],
             [9, 10, 15, 8]]

start37_2 = [[7, 11, 12, 4],
             [2, 3, 8, 14],
             [6, 0, 1, 15],
             [9, 5, 10, 13]]
# your code

#### Task 3.2

Explain why there is a difference between the number of the expanded nodes for these starting states if their costs of the optimal paths are the same.

### Part 4:  Constraint Satisfaction Problem [5 marks]

This part of the assignment is based on another puzzle called Sudoku. The game consists of 9x9 grid with a few cells filled in with digits. The objective of the game is to fill the remaining cells so that each column, each row, and each of the nine 3×3 sub-grids contain all of the digits from 1 to 9.

The start state of the puzzle is defined as a multidimensional array of numbers, where 0 represents empty cells, for example:

In [40]:
grid = [[9, 5, 0, 8, 2, 7, 3, 0, 0],
        [0, 8, 0, 1, 4, 0, 0, 5, 0],
        [0, 1, 0, 5, 9, 0, 0, 0, 0],
        [8, 3, 0, 0, 0, 0, 0, 7, 5],
        [1, 6, 9, 7, 5, 2, 4, 3, 0],
        [0, 7, 0, 0, 8, 0, 0, 6, 0],
        [0, 9, 1, 0, 6, 0, 8, 4, 0],
        [7, 0, 8, 0, 3, 1, 0, 0, 6],
        [6, 2, 0, 4, 7, 8, 0, 9, 0]]

*Tip: If you have problems with understanding how to implement constrains take a look at cspExamples.py and cspExamplesQueens.py*

#### Task 4.1

Write the code that defines constraint(s) and a function grid_to_csp that returns a CSP for a Sudoku puzzle:

In [41]:
# define constraint function(s)


def isValid(grid, i, j, e):
    validRow = all([e != grid[i][x] for x in range(9)])
    if validRow:
        validCol = all([e != grid[x][j] for x in range(9)])
        if validCol:
            # finding the top left x,y co-ordinates of the section containing the i,j cell
            secTopX, secTopY = 3 *(i//3), 3 *(j//3)
            for x in range(secTopX, secTopX+3):
                for y in range(secTopY, secTopY+3):
                    if grid[x][y] == e:
                        return False
            return True
    return False

# function that returns a csp
def grid_to_csp(grid):
    domains = {1,2,3,4,5,6,7,8,9}
    constraints = [

    ]
    return CSP(domains, constraints)

# csp
csp = grid_to_csp(grid)

#### Task 4.2

Write the code that solves Sudoku csp using a search algorithms of your choice, print the solution.

In [47]:
# your code

def findNextFill(grid, i, j):
    for x in range(i,9):
        for y in range(j,9):
            if grid[x][y] == 0:
                return x,y
    for x in range(0,9):
        for y in range(0,9):
            if grid[x][y] == 0:
                return x,y
    return -1,-1

def isValid(grid, i, j, e):
    validRow = all([e != grid[i][x] for x in range(9)])
    if validRow:
        validCol = all([e != grid[x][j] for x in range(9)])
        if validCol:
            # finding the top left x,y co-ordinates of the section containing the i,j cell
            secTopX, secTopY = 3 *(i//3), 3 *(j//3)
            for x in range(secTopX, secTopX+3):
                for y in range(secTopY, secTopY+3):
                    if grid[x][y] == e:
                        return False
            return True
    return False

def solve(grid, i=0, j=0):
    i,j = findNextFill(grid, i, j)
    if i == -1:
        return True
    for e in range(1,10):
        if isValid(grid,i,j,e):
            grid[i][j] = e
            if solve(grid, i, j):
                    return True
            # Undo the current cell for backtracking
            grid[i][j] = 0
    return False

solve(grid)
for i in grid:
    print(i)

[9, 5, 6, 8, 2, 7, 3, 1, 4]
[2, 8, 3, 1, 4, 6, 7, 5, 9]
[4, 1, 7, 5, 9, 3, 6, 8, 2]
[8, 3, 2, 6, 1, 4, 9, 7, 5]
[1, 6, 9, 7, 5, 2, 4, 3, 8]
[5, 7, 4, 3, 8, 9, 2, 6, 1]
[3, 9, 1, 2, 6, 5, 8, 4, 7]
[7, 4, 8, 9, 3, 1, 5, 2, 6]
[6, 2, 5, 4, 7, 8, 1, 9, 3]


#### Task 4.3

Descibe what do nodes represent in the search tree. Explain your choice of the search algorithm and compare it with any other search algorithm implemented in this assignemnt. 